## Importação de Bibliotecas

Carrega todas as dependências necessárias para operações PySpark: funções de agregação, transformação, window functions e tipos de dados Delta Lake.
Nota: `F` é importado como alias para operações SQL avançadas; `month`, `dayofmonth` permitem granularidade temporal em análises de satisfação.

In [0]:
from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import (
    col, count, avg, sum as spark_sum, max as spark_max, min as spark_min,
    current_timestamp, lit, when, round as spark_round, to_date, weekofyear, year,
    month, dayofmonth,
    row_number, percentile_approx, collect_list, struct, countDistinct
)
from pyspark.sql.types import IntegerType, StringType, TimestampType, DateType, FloatType, DecimalType
from pyspark.sql.window import Window
from pyspark.sql.utils import AnalysisException
import pyspark.sql.functions as F

## Configuração do Catálogo Unity

Define constantes para navegação na arquitetura Medallion (atendimento_catalog → silver/gold).
Estas variáveis estabelecem os namespaces padrão: `silver` como origem dos dados (leitura) e `gold` como destino (escrita).

In [0]:
catalog_name = "atendimento_catalog"
silver_db_name = "silver"
gold_db_name = "gold"

## Ativação do Catálogo e Schema

Configura o contexto Spark para usar o catálogo `atendimento_catalog` e o schema `gold`.
Todos os comandos SQL e operações de escrita subsequentes utilizarão automaticamente este namespace, evitando ambiguidade.

In [0]:
spark.sql(f"CREATE CATALOG IF NOT EXISTS {catalog_name}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {gold_db_name}")
spark.sql(f"USE CATALOG {catalog_name}")
spark.sql(f"USE SCHEMA {gold_db_name}")

## Funções Auxiliares para Validação

`safe_table_exists()` verifica se uma tabela existe sem lançar exceção AnalysisException.
`safe_col()` retorna uma coluna existente ou NULL tipado se ausente.
Ambas garantem robustez em cenários onde dimensões podem estar temporariamente indisponíveis.

## Funções Utilitárias

Funções auxiliares para validação e transformação de dados.

In [0]:
def safe_table_exists(full_name: str) -> bool:
    try:
        spark.table(full_name)
        return True
    except AnalysisException:
        return False
    except Exception:
        return False

def safe_col(df, name):
    return col(name) if name in df.columns else lit(None).cast(StringType())

## Leitura das Tabelas Silver

**Obrigatória:** `ft_chamados` (tabela fato de chamados)  
**Opcionais:** `dm_clientes`, `dm_canais`, `dm_motivos`, `ft_atendentes`, `ft_custos`, `ft_pesquisa_satisfacao`  

Cada tabela é carregada em memória com verificação prévia de existência. O pipeline tolera dimensões ausentes (retorna None), resultando em colunas NULL nas agregações downstream.
Contagem de registros na origem (`total_before_chamados`) é registrada para auditoria.

In [0]:
src_table_chamados = f"{catalog_name}.{silver_db_name}.ft_chamados"
src_table_clientes = f"{catalog_name}.{silver_db_name}.dm_clientes"
src_table_canais = f"{catalog_name}.{silver_db_name}.dm_canais"
src_table_motivos = f"{catalog_name}.{silver_db_name}.dm_motivos"
src_table_atendentes = f"{catalog_name}.{silver_db_name}.ft_atendentes"
src_table_custos = f"{catalog_name}.{silver_db_name}.ft_custos"
src_table_satisfacao = f"{catalog_name}.{silver_db_name}.ft_pesquisa_satisfacao"
tgt_table_fact = f"{catalog_name}.{gold_db_name}.fact_chamados"

if not safe_table_exists(src_table_chamados):
    raise RuntimeError(f"Tabela fonte não encontrada: {src_table_chamados}")

df_chamados_src = spark.table(src_table_chamados)
df_clientes_src = spark.table(src_table_clientes) if safe_table_exists(src_table_clientes) else None
df_canais_src = spark.table(src_table_canais) if safe_table_exists(src_table_canais) else None
df_motivos_src = spark.table(src_table_motivos) if safe_table_exists(src_table_motivos) else None
df_atendentes_src = spark.table(src_table_atendentes) if safe_table_exists(src_table_atendentes) else None
df_custos_src = spark.table(src_table_custos) if safe_table_exists(src_table_custos) else None
df_satisfacao_src = spark.table(src_table_satisfacao) if safe_table_exists(src_table_satisfacao) else None

total_before_chamados = df_chamados_src.count()

print(f"Leitura: {src_table_chamados}")
print(f"Registros ft_chamados: {total_before_chamados:,}")

## Construção da Tabela Fato Principal

Enriquece a fato de chamados com dimensões via left joins graceful (cada join opcional).

**Campos adicionados:**
- Demográficos: `regiao` (cliente)
- Operacionais: `canal`, `motivo`, `nome_atendente`, `nivel_atendimento`
- Financeiros: `custo` (com flag de auditoria `flag_custo_ausente`)
- Qualidade: `nota_atendimento` (com flag `flag_satisfacao_ausente`)
- Temporais: `data`, `semana`, `ano`

Mantém granularidade original (1 registro = 1 chamado). Flags de auditoria habilitam rastreabilidade de cobertura—crítico para análises de ROI e satisfação.
Todos os campos recebem casting explícito para garantir consistência de tipos nas aggregações downstream.

In [0]:
df_fact = df_chamados_src

if df_clientes_src is not None:
    df_fact = df_fact.alias("ch").join(
        df_clientes_src.alias("cl"),
        col("ch.id_cliente") == col("cl.id_cliente"),
        "left"
    ).select(
        col("ch.*"),
        col("cl.regiao")
    )
else:
    df_fact = df_fact.withColumn("regiao", lit(None).cast(StringType()))

if df_canais_src is not None:
    df_fact = df_fact.alias("f").join(
        df_canais_src.alias("ca"),
        col("f.id_canal") == col("ca.id_canal"),
        "left"
    ).select("f.*", col("ca.id_canal").alias("id_canal_dim"))
else:
    df_fact = df_fact.withColumn("id_canal_dim", lit(None).cast(IntegerType()))

if df_motivos_src is not None:
    df_fact = df_fact.alias("f").join(
        df_motivos_src.alias("m"),
        col("f.id_motivo") == col("m.id_motivo"),
        "left"
    ).select("f.*")
else:
    pass

if df_atendentes_src is not None:
    df_fact = df_fact.alias("f").join(
        df_atendentes_src.alias("a"),
        col("f.id_atendente") == col("a.id_atendente"),
        "left"
    ).select("f.*")
else:
    pass

if df_custos_src is not None:
    df_fact = df_fact.alias("f").join(
        df_custos_src.alias("cu"),
        col("f.id_chamado") == col("cu.id_chamado"),
        "left"
    ).select(
        col("f.*"),
        col("cu.custo")
    )
else:
    df_fact = df_fact.withColumn("custo", lit(None))

if df_satisfacao_src is not None:
    df_fact = df_fact.alias("f").join(
        df_satisfacao_src.alias("s"),
        col("f.id_chamado") == col("s.id_chamado"),
        "inner"
    ).select(
        col("f.*"),
        col("s.nota_atendimento")
    )
else:
    df_fact = df_fact.withColumn("nota_atendimento", lit(None).cast(IntegerType()))

df_fact = df_fact.withColumn("flag_custo_ausente", 
    when(col("custo").isNull(), lit(1)).otherwise(lit(0)).cast(IntegerType())
)
df_fact = df_fact.withColumn("flag_satisfacao_ausente", 
    when(col("nota_atendimento").isNull(), lit(1)).otherwise(lit(0)).cast(IntegerType())
)

df_fact = df_fact.withColumn("data", to_date(col("hora_abertura_chamado")))
df_fact = df_fact.withColumn("semana", weekofyear(col("hora_abertura_chamado")))
df_fact = df_fact.withColumn("ano", year(col("hora_abertura_chamado")))
df_fact = df_fact.withColumn("processed_timestamp", current_timestamp())

df_fact_typed = df_fact \
    .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("id_atendente", col("id_atendente").cast(IntegerType())) \
    .withColumn("id_motivo", col("id_motivo").cast(IntegerType())) \
    .withColumn("id_canal", col("id_canal").cast(IntegerType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("resolvido", col("resolvido").cast(StringType())) \
    .withColumn("tempo_espera_minutos", col("tempo_espera_minutos").cast(FloatType())) \
    .withColumn("tempo_atendimento_minutos", col("tempo_atendimento_minutos").cast(FloatType())) \
    .withColumn("custo", col("custo").cast(DecimalType(18, 8))) \
    .withColumn("nota_atendimento", col("nota_atendimento").cast(IntegerType())) \
    .withColumn("flag_custo_ausente", col("flag_custo_ausente").cast(IntegerType())) \
    .withColumn("flag_satisfacao_ausente", col("flag_satisfacao_ausente").cast(IntegerType())) \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

df_fact_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_fact)

fact_chamados = spark.table(tgt_table_fact)
final_count_fact = fact_chamados.count()

print(f"Salvo em: {tgt_table_fact}")
print(f"Registros na tabela fato: {final_count_fact:,}")

## Auditoria de Cobertura de Dados

Valida os flags de auditoria criados na fato: percentual de chamados com custo presente e satisfação respondida.
**Aviso:** Cobertura < 95% em qualquer métrica indica problema na coleta de dados Silver—esclareça com origem antes de prosseguir análises financeiras ou de NPS.

In [0]:
total_chamados = fact_chamados.count()
sem_custo = fact_chamados.filter(col("flag_custo_ausente") == 1).count()
sem_satisfacao = fact_chamados.filter(col("flag_satisfacao_ausente") == 1).count()

perc_cobertura_custo = ((total_chamados - sem_custo) / total_chamados * 100) if total_chamados > 0 else 0
perc_cobertura_satisfacao = ((total_chamados - sem_satisfacao) / total_chamados * 100) if total_chamados > 0 else 0

print("="*80)
print("RELATÓRIO DE AUDITORIA - FACT_CHAMADOS")
print("="*80)
print(f"Total de chamados: {total_chamados:,}")
print(f"Chamados sem custo: {sem_custo:,} ({100-perc_cobertura_custo:.2f}%)")
print(f"Chamados sem nota de satisfação: {sem_satisfacao:,} ({100-perc_cobertura_satisfacao:.2f}%)")
print(f"Cobertura de custo: {perc_cobertura_custo:.2f}%")
print(f"Cobertura de satisfação: {perc_cobertura_satisfacao:.2f}%")
print("="*80)
print("✓ Flags de auditoria disponíveis para rastreabilidade em análises Gold")
print("="*80)

## Agregação: Métricas de Funil por Dimensão

Agrega chamados por `data`, `semana`, `ano`, `canal` e `regiao`.

**Métricas calculadas:**
- `n_autosservico`: Chamados resolvidos sem atendente (critério: `id_atendente IS NULL AND resolvido='Sim'`) ← **Correção crítica aplicada**
- `n_nivel1`: Chamados que tiveram atendimento humano
- `n_resolvido`: Total resolvido (independente de canal)
- `volume_total_chamados`: Total de chamados no período/dimensão
- `tempo_*_medio`: Médias de espera e atendimento
- `custo_total`: Custo agregado do período

Esta tabela alimenta análises de eficiência de autosserviço vs atendimento humano.

In [0]:
tgt_table_funnel = f"{catalog_name}.{gold_db_name}.funnel_metrics"

df_funnel = fact_chamados.withColumn(
    "n_autosservico",
    when(
        (col("id_atendente").isNull()) & 
        (col("resolvido") == "Sim"),
        1
    ).otherwise(0).cast(IntegerType())
).withColumn(
    "n_nivel1",
    when(col("id_atendente").isNotNull(), 1).otherwise(0).cast(IntegerType())
).withColumn(
    "n_resolvido",
    when(col("resolvido") == "Sim", 1).otherwise(0).cast(IntegerType())
)

df_funnel_agg = df_funnel.groupBy("data", "semana", "ano", "canal", "regiao").agg(
    spark_sum("n_autosservico").alias("n_autosservico"),
    spark_sum("n_nivel1").alias("n_nivel1"),
    spark_sum("n_resolvido").alias("n_resolvido"),
    count("id_chamado").alias("volume_total_chamados"),
    spark_round(avg("tempo_espera_minutos"), 2).cast(DecimalType(10, 2)).alias("tempo_espera_medio"),
    spark_round(avg("tempo_atendimento_minutos"), 2).cast(DecimalType(10, 2)).alias("tempo_atendimento_medio"),
    spark_round(spark_sum("custo"), 2).cast(DecimalType(10, 2)).alias("custo_total")
)

df_funnel_agg = df_funnel_agg.withColumn("processed_timestamp", current_timestamp())

df_funnel_typed = df_funnel_agg \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("n_autosservico", col("n_autosservico").cast(IntegerType())) \
    .withColumn("n_nivel1", col("n_nivel1").cast(IntegerType())) \
    .withColumn("n_resolvido", col("n_resolvido").cast(IntegerType())) \
    .withColumn("volume_total_chamados", col("volume_total_chamados").cast(IntegerType())) \
    .withColumn("tempo_espera_medio", col("tempo_espera_medio").cast(DecimalType(10, 2))) \
    .withColumn("tempo_atendimento_medio", col("tempo_atendimento_medio").cast(DecimalType(10, 2))) \
    .withColumn("custo_total", col("custo_total").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

df_funnel_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_funnel)

gold_table_funnel = spark.table(tgt_table_funnel)
final_count_funnel = gold_table_funnel.count()

print(f"Salvo em: {tgt_table_funnel}")
print(f"Registros agregados: {final_count_funnel:,}")

## Relatório de Transformação: Contagem antes/depois

Compara cardinalidade de entrada (Silver `ft_chamados`) com saída agregada (`funnel_metrics`).
**Diagnóstico:** Redução grande é esperada (muitos chamados → poucas dimensões de agregação). Contagem zero em Gold sugere problema em join ou filtro.

In [0]:
print("="*80)
print("RELATÓRIO DE TRANSFORMAÇÃO - FUNNEL_METRICS")
print("="*80)
print(f"Registros Silver (origem chamados): {total_before_chamados:,}")
print(f"Registros Gold (destino): {final_count_funnel:,}")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

## Detecção de Escalações Anômalas

Identifica chamados que iniciaram em canais digitais (Ura, Chatbot, Web) mas precisaram de atendente humano.
Estas são escalações "anômalas" que indicam falha do autosserviço.

### Propósito
**Entrada:** Tabela Gold `fact_chamados`  
**Saída:** Tabela Gold `anomalous_escalations`  
**Resumo da lógica:** Identifica escalações anômalas: chamados iniciados em canais digitais (Ura/Chatbot/Web) que precisaram de atendente humano (`id_atendente IS NOT NULL`). Adiciona flag `tipo_anomalia` descritiva.  
**Observações:** Correção crítica aplicada: filtro invertido corrigido. Agora detecta corretamente falhas de autosserviço (canal digital → escalação humana), não todas as escalações humanas.

In [0]:
tgt_table_anomalous = f"{catalog_name}.{gold_db_name}.anomalous_escalations"

df_anomalous = fact_chamados.filter(
    (col("id_atendente").isNotNull()) &
    (col("canal").isin(["Ura", "Chatbot", "Web"]))
).select(
    col("id_chamado"),
    col("id_cliente"),
    col("data"),
    col("canal"),
    col("id_motivo"),
    col("motivo"),
    col("resolvido"),
    lit("Escalação de canal digital para atendente humano").alias("tipo_anomalia")
).withColumn("processed_timestamp", current_timestamp())

df_anomalous_typed = df_anomalous \
    .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("id_motivo", col("id_motivo").cast(IntegerType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("tipo_anomalia", col("tipo_anomalia").cast(StringType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de anomalias detectadas: {df_anomalous_typed.count():,}")

df_anomalous_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_anomalous)

gold_table_anomalous = spark.table(tgt_table_anomalous)
final_count_anomalous = gold_table_anomalous.count()

print(f"Salvo em: {tgt_table_anomalous}")
print(f"Contagem final: {final_count_anomalous:,}")

## Análise: Motivos mais Frequentes

Agrega chamados por motivo e conta volume total.
Ordenação descendente (top-first) destaca as demandas mais comuns—útil para priorizar treinamento de equipes e melhorias operacionais.

In [0]:
tgt_table_top_volume = f"{catalog_name}.{gold_db_name}.top_reasons_volume"

df_top_volume = fact_chamados.groupBy("id_motivo", "motivo").agg(
    count("id_chamado").alias("volume_chamados")
).orderBy(col("volume_chamados").desc())

df_top_volume = df_top_volume.withColumn("processed_timestamp", current_timestamp())

df_top_volume_typed = df_top_volume \
    .withColumn("id_motivo", col("id_motivo").cast(IntegerType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("volume_chamados", col("volume_chamados").cast(IntegerType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_volume_typed.count():,}")

## Persistência e Validação

Grava em Delta com `overwriteSchema=true` para permitir evolução de schema sem erro.
Exibe top 10 motivos para validação visual rápida.

In [0]:
df_top_volume_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_volume)

gold_table_top_volume = spark.table(tgt_table_top_volume)
final_count_top_volume = gold_table_top_volume.count()

print(f"Salvo em: {tgt_table_top_volume}")
print(f"Contagem final: {final_count_top_volume:,}")

display(gold_table_top_volume.limit(10))

## Análise: Custos por Motivo

Agrega `custo_total` e `volume_chamados` por motivo, calcula `custo_medio` (total ÷ volume).
`Custo_medio` permite comparação justa entre motivos de volumes diferentes—essencial para identificar oportunidades de redução de custo por tipo de demanda.
Ordenação: motivos mais caros primeiro.

In [0]:
tgt_table_top_cost = f"{catalog_name}.{gold_db_name}.top_reasons_cost"

df_top_cost = fact_chamados.groupBy("id_motivo", "motivo").agg(
    spark_round(spark_sum("custo"), 2).alias("custo_total"),
    count("id_chamado").alias("volume_chamados")
).orderBy(col("custo_total").desc())

df_top_cost = df_top_cost.withColumn(
    "custo_medio",
    spark_round(
        when(
            col("volume_chamados") > 0,
            col("custo_total") / col("volume_chamados")
        ).otherwise(lit(None)),
        2
    )
)

df_top_cost = df_top_cost.withColumn("processed_timestamp", current_timestamp())

df_top_cost_typed = df_top_cost \
    .withColumn("id_motivo", col("id_motivo").cast(IntegerType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("custo_total", col("custo_total").cast(DecimalType(10, 2))) \
    .withColumn("volume_chamados", col("volume_chamados").cast(IntegerType())) \
    .withColumn("custo_medio", col("custo_medio").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))


print(f"Total de motivos únicos: {df_top_cost_typed.count():,}")

## Persistência e Visualização

Grava em Delta e exibe top 10 motivos ordenados por custo decrescente.
Display permite validação visual imediata de cálculos financeiros.

In [0]:
df_top_cost_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_cost)

gold_table_top_cost = spark.table(tgt_table_top_cost)
final_count_top_cost = gold_table_top_cost.count()

print(f"Salvo em: {tgt_table_top_cost}")
print(f"Contagem final: {final_count_top_cost:,}")

display(gold_table_top_cost.limit(10))

## Análise: Motivos com Baixa Satisfação

Agrega por motivo: `nota_media`, `volume_respostas`, `count_baixa_satisfacao` (nota ≤ 3), e `perc_baixa_satisfacao`.
Ordenação ascendente por nota média—motivos críticos (pior satisfação) primeiro.
**Definição:** Baixa satisfação = nota ≤ 3 (escala 1–5 típica). Percentual permite comparação entre motivos de volumes diferentes.
Alerta: Motivos com `perc_baixa_satisfacao` > 50% requerem ação imediata (redesenho de atendimento, treinamento).

In [0]:
tgt_table_top_satisfaction = f"{catalog_name}.{gold_db_name}.top_reasons_low_satisfaction"

df_base_satisfaction = fact_chamados.filter(
    col("nota_atendimento").isNotNull()
).select(
    col("id_chamado"),
    col("id_motivo"),
    col("motivo"),
    col("nota_atendimento")
)

print(f"Registros com pesquisa de satisfação: {df_base_satisfaction.count():,}")

## Preparação: Base de Satisfação por Motivo

Filtra `fact_chamados` para apenas registros com `nota_atendimento IS NOT NULL` (chamados com pesquisa respondida).
**Correção aplicada:** Usa `nota_atendimento` já presente em fact_chamados (eliminando join redundante com Silver `ft_pesquisa_satisfacao`).
Garante consistência temporal—uma nota por chamado, sem risco de duplicação.

In [0]:
df_top_satisfaction = df_base_satisfaction.groupBy("id_motivo", "motivo").agg(
    spark_round(avg("nota_atendimento"), 2).alias("nota_media"),
    count("id_chamado").alias("volume_respostas"),
    spark_sum(
        when(col("nota_atendimento") <= 3, 1).otherwise(0)
    ).alias("count_baixa_satisfacao")
).orderBy(col("nota_media").asc())

df_top_satisfaction = df_top_satisfaction.withColumn(
    "perc_baixa_satisfacao",
    spark_round(
        when(
            col("volume_respostas") > 0,
            (col("count_baixa_satisfacao") / col("volume_respostas")) * 100
        ).otherwise(lit(0.0)),
        2
    )
)

df_top_satisfaction = df_top_satisfaction.withColumn("processed_timestamp", current_timestamp())

df_top_satisfaction_typed = df_top_satisfaction \
    .withColumn("id_motivo", col("id_motivo").cast(IntegerType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("nota_media", col("nota_media").cast(DecimalType(10, 2))) \
    .withColumn("volume_respostas", col("volume_respostas").cast(IntegerType())) \
    .withColumn("count_baixa_satisfacao", col("count_baixa_satisfacao").cast(IntegerType())) \
    .withColumn("perc_baixa_satisfacao", col("perc_baixa_satisfacao").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de motivos únicos: {df_top_satisfaction_typed.count():,}")


## Persistência e Visualização

Grava em Delta e exibe top 10 motivos ordenados por nota média crescente (piores no topo).
Display facilita identificação rápida de crises de satisfação.

In [0]:
df_top_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_top_satisfaction)

gold_table_top_satisfaction = spark.table(tgt_table_top_satisfaction)
final_count_top_satisfaction = gold_table_top_satisfaction.count()

print(f"Salvo em: {tgt_table_top_satisfaction}")
print(f"Contagem final: {final_count_top_satisfaction:,}")

display(gold_table_top_satisfaction.limit(10))

## Análise: Tendências de Satisfação Temporal

Agrega satisfação por `data`, `semana`, `ano`, `canal`, `regiao`.

**Métricas:**
- `nota_media`: Nota média de satisfação (escala 1–5)
- `perc_satisfeito`: % de chamados com nota ≥ 4
- `count_insatisfeito`: Count de nota ≤ 3 (rastreamento absoluto)
- `total_respostas`: Denominator para cálculo de percentuais

**Correção aplicada:** Usa `nota_atendimento` de `fact_chamados` diretamente (não join com Silver `ft_pesquisa_satisfacao`)—garante 1:1 mapping chamado→nota.
Granularidade temporal múltipla (data + semana + ano) permite análises em diferentes níveis—tendências diárias vs semanais vs anuais.

In [0]:
tgt_table_satisfaction_trends = f"{catalog_name}.{gold_db_name}.satisfaction_trends"

df_satisfaction_full = fact_chamados.filter(
    col("nota_atendimento").isNotNull()
).select(
    col("data"),
    col("semana"),
    col("ano"),
    col("canal"),
    col("regiao"),
    col("nota_atendimento")
)

df_satisfaction = df_satisfaction_full.groupBy("data", "semana", "ano", "canal", "regiao").agg(
    spark_round(avg("nota_atendimento"), 2).alias("nota_media"),
    count("nota_atendimento").alias("total_respostas"),
    spark_sum(when(col("nota_atendimento") >= 4, 1).otherwise(0)).alias("count_satisfeito"),
    spark_sum(when(col("nota_atendimento") <= 3, 1).otherwise(0)).alias("count_insatisfeito")
)

df_satisfaction = df_satisfaction.withColumn(
    "perc_satisfeito",
    spark_round(
        when(
            col("total_respostas") > 0,
            (col("count_satisfeito") / col("total_respostas")) * 100
        ).otherwise(lit(0.0)),
        2
    )
)

df_satisfaction = df_satisfaction.withColumn("processed_timestamp", current_timestamp())

df_satisfaction_typed = df_satisfaction \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("nota_media", col("nota_media").cast(DecimalType(10, 2))) \
    .withColumn("total_respostas", col("total_respostas").cast(IntegerType())) \
    .withColumn("count_satisfeito", col("count_satisfeito").cast(IntegerType())) \
    .withColumn("count_insatisfeito", col("count_insatisfeito").cast(IntegerType())) \
    .withColumn("perc_satisfeito", col("perc_satisfeito").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de registros: {df_satisfaction_typed.count():,}")


## Persistência e Visualização

Grava em Delta e exibe primeiros 10 registros ordenados por data.
Esta tabela alimenta dashboards de evolução temporal de satisfação por canal/região—identifica tendências e sazonalidades.

In [0]:
df_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_satisfaction_trends)

gold_table_satisfaction_trends = spark.table(tgt_table_satisfaction_trends)
final_count_satisfaction_trends = gold_table_satisfaction_trends.count()

print(f"Salvo em: {tgt_table_satisfaction_trends}")
print(f"Contagem final: {final_count_satisfaction_trends:,}")

display(gold_table_satisfaction_trends.limit(10))

## Perfis de Clientes: Comportamento Agregado

Agrega por cliente (`id_cliente`, `regiao`):
- `total_chamados`, `total_resolvidos`, `taxa_resolucao` (taxa em %)  
- `tempo_espera_medio`, `custo_total_cliente`  
- `motivo_principal`: Motivo mais frequente para aquele cliente (via window function `row_number()`)

**Aplicação:** Segmentar clientes por complexidade (taxa de resolução) e padrão de demanda (motivo principal).
Útil para personalização de atendimento, priorização de campanhas de training, e análise de rentabilidade por cliente.

In [0]:
tgt_table_client_profiles = f"{catalog_name}.{gold_db_name}.client_profiles"

w_motivo = Window.partitionBy("id_cliente").orderBy(col("count_motivo").desc())

df_motivo_por_cliente = fact_chamados.groupBy("id_cliente", "id_motivo", "motivo").agg(
    count("id_chamado").alias("count_motivo")
)

df_motivo_principal = df_motivo_por_cliente \
    .withColumn("rn", row_number().over(w_motivo)) \
    .filter(col("rn") == 1) \
    .select(col("id_cliente"), col("id_motivo"), col("motivo").alias("motivo_principal"))

df_client_profiles = fact_chamados.groupBy("id_cliente", "regiao").agg(
    count("id_chamado").alias("total_chamados"),
    spark_sum(when(col("resolvido") == "Sim", 1).otherwise(0)).alias("total_resolvidos"),
    spark_round(avg("tempo_espera_minutos"), 2).alias("tempo_espera_medio"),
    spark_round(spark_sum("custo"), 2).alias("custo_total_cliente")
)

df_client_profiles = df_client_profiles.join(df_motivo_principal, "id_cliente", "left")

df_client_profiles = df_client_profiles.withColumn(
    "taxa_resolucao",
    spark_round(
        when(
            col("total_chamados") > 0,
            (col("total_resolvidos") / col("total_chamados")) * 100
        ).otherwise(lit(0.0)),
        2
    )
)

df_client_profiles = df_client_profiles.withColumn("processed_timestamp", current_timestamp())

df_client_profiles_typed = df_client_profiles \
    .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
    .withColumn("regiao", col("regiao").cast(StringType())) \
    .withColumn("total_chamados", col("total_chamados").cast(IntegerType())) \
    .withColumn("total_resolvidos", col("total_resolvidos").cast(IntegerType())) \
    .withColumn("tempo_espera_medio", col("tempo_espera_medio").cast(DecimalType(10, 2))) \
    .withColumn("custo_total_cliente", col("custo_total_cliente").cast(DecimalType(10, 2))) \
    .withColumn("id_motivo", col("id_motivo").cast(IntegerType())) \
    .withColumn("motivo_principal", col("motivo_principal").cast(StringType())) \
    .withColumn("taxa_resolucao", col("taxa_resolucao").cast(DecimalType(10, 2))) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))


## Persistência e Validação

Grava em Delta e exibe top 10 clientes (por volume de chamados).
Display permite validação visual: outliers com volume ou custo extremo indicam clientes críticos ou anomalias de dados.

In [0]:
df_client_profiles_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_client_profiles)

gold_table_client_profiles = spark.table(tgt_table_client_profiles)
final_count_client_profiles = gold_table_client_profiles.count()

print(f"Salvo em: {tgt_table_client_profiles}")
print(f"Contagem final: {final_count_client_profiles:,}")

display(gold_table_client_profiles.limit(10))

## Dataset para Análises de Correlação

Extrai features de chamados com satisfação respondida: `resolvido`, `tempo_espera_minutos`, `tempo_atendimento_minutos`, `nota_atendimento`.
Adiciona granularidade temporal (`ano`, `mes`, `dia`, `semana`) para estratificação.

**Uso:** Análises estatísticas (correlação Pearson/Spearman entre tempos e satisfação).
**Correção aplicada:** Usa `nota_atendimento` de `fact_chamados` (não join com Silver)—garante consistência 1:1 e rastreabilidade temporal.

In [0]:
tgt_table_satisfaction_correlations = f"{catalog_name}.{gold_db_name}.ft_correlacao_satisfacao"

df_satisfaction_correlations = fact_chamados.filter(
    col("nota_atendimento").isNotNull()
).select(
    col("id_chamado"),
    col("resolvido"),
    col("tempo_espera_minutos"),
    col("tempo_atendimento_minutos"),
    col("nota_atendimento"),
    col("data")
).withColumn("ano", year("data")) \
 .withColumn("mes", month("data")) \
 .withColumn("dia", dayofmonth("data")) \
 .withColumn("semana", weekofyear("data")) \
 .withColumn("processed_timestamp", current_timestamp())

df_satisfaction_correlations_typed = df_satisfaction_correlations \
    .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
    .withColumn("resolvido", col("resolvido").cast(StringType())) \
    .withColumn("tempo_espera_minutos", col("tempo_espera_minutos").cast(DecimalType(10, 2))) \
    .withColumn("tempo_atendimento_minutos", col("tempo_atendimento_minutos").cast(DecimalType(10, 2))) \
    .withColumn("nota_atendimento", col("nota_atendimento").cast(IntegerType())) \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("mes", col("mes").cast(IntegerType())) \
    .withColumn("dia", col("dia").cast(IntegerType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

print(f"Total de registros com satisfação: {df_satisfaction_correlations_typed.count():,}")

## Persistência e Validação

Grava em Delta e exibe primeiros 10 registros.
Dataset pronto para consumo em ferramentas estatísticas (Python/R) ou análises avançadas in-notebook (correlação, regressão).

In [0]:
df_satisfaction_correlations_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_satisfaction_correlations)

gold_table_satisfaction_correlations = spark.table(tgt_table_satisfaction_correlations)
final_count_satisfaction_correlations = gold_table_satisfaction_correlations.count()

print(f"Salvo em: {tgt_table_satisfaction_correlations}")
print(f"Contagem final: {final_count_satisfaction_correlations:,}")

display(gold_table_satisfaction_correlations.limit(10))

## Análise: Satisfação por Canal ao Longo do Tempo

Agrega por `id_canal`, `canal` e granularidade temporal (`data`, `semana`, `ano`).
Calcula `nota_media` e métricas de satisfação/insatisfação por canal-período.

**Correção aplicada:** Usa `nota_atendimento` de `fact_chamados` (não join com Silver)—garante 1:1 mapping.
**Aplicação:** Identifica canais com tendência crescente de insatisfação (degrado de qualidade, evolução de demanda).
Útil para ajuste de SLAs por canal e alocação de investimento em melhorias.

In [0]:
tgt_table_channel_satisfaction = f"{catalog_name}.{gold_db_name}.ft_canal_satisfacao"

df_channel_satisfaction = fact_chamados.filter(
    col("nota_atendimento").isNotNull()
).select(
    col("id_chamado"),
    col("id_canal"),
    col("canal"),
    col("nota_atendimento"),
    col("data")
).withColumn("ano", year("data")) \
 .withColumn("mes", month("data")) \
 .withColumn("dia", dayofmonth("data")) \
 .withColumn("semana", weekofyear("data")) \
 .withColumn("processed_timestamp", current_timestamp())

df_channel_satisfaction_typed = df_channel_satisfaction \
    .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
    .withColumn("id_canal", col("id_canal").cast(IntegerType())) \
    .withColumn("canal", col("canal").cast(StringType())) \
    .withColumn("nota_atendimento", col("nota_atendimento").cast(IntegerType())) \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("mes", col("mes").cast(IntegerType())) \
    .withColumn("dia", col("dia").cast(IntegerType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

df_channel_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_channel_satisfaction)

gold_table_channel_satisfaction = spark.table(tgt_table_channel_satisfaction)
final_count_channel_satisfaction = gold_table_channel_satisfaction.count()

print(f"Salvo em: {tgt_table_channel_satisfaction}")
print(f"Contagem final: {final_count_channel_satisfaction:,}")

display(gold_table_channel_satisfaction.limit(10))

## Performance de Atendentes: Métricas Individuais

Agrega por atendente (`id_atendente`, `nome_atendente`, `nivel_atendimento`):
- `total_chamados`, `total_resolvidos`, `taxa_resolucao` (%)  
- `nota_media_atendente`: Média de satisfação para aquele atendente

Dados obtidos via join com `ft_atendentes` Silver (nome, nível), agregação com `fact_chamados` (volume, resolução), e subquery de nota média.

**Aplicação:** Identificar atendentes com performance crítica (taxa < 70% ou nota < 3.0)—candidatos a treinamento, coaching, realocação.
**Dependência:** Requer `ft_atendentes` disponível.

In [0]:
tgt_table_chamados_por_atendente = f"{catalog_name}.{gold_db_name}.chamados_por_atendente"

if df_atendentes_src is not None:
    df_chamados_com_atendente = fact_chamados.join(
        df_atendentes_src, 
        fact_chamados["id_atendente"] == df_atendentes_src["id_atendente"],
        "inner"
    ).select(
        fact_chamados["*"],
        df_atendentes_src["nome_atendente"],
        df_atendentes_src["nivel_atendimento"]
    )

    df_nota_media = fact_chamados.filter(
        col("nota_atendimento").isNotNull()
    ).groupBy("id_atendente").agg(
        F.round(F.avg("nota_atendimento"), 2).alias("nota_media_atendente")
    )

    df_chamados_por_atendente = df_chamados_com_atendente.groupBy(
        "id_atendente", "nome_atendente", "nivel_atendimento"
    ).agg(
        F.count("id_chamado").alias("total_chamados"),
        F.sum(F.when(F.col("resolvido") == "Sim", 1).otherwise(0)).alias("total_resolvidos")
    )

    df_chamados_por_atendente = df_chamados_por_atendente.join(
        df_nota_media, "id_atendente", "left"
    )

    df_chamados_por_atendente = df_chamados_por_atendente.withColumn(
        "taxa_resolucao",
        F.round(F.col("total_resolvidos") * 100 / F.col("total_chamados"), 2)
    ).withColumn("processed_timestamp", F.current_timestamp())

    df_chamados_por_atendente_typed = df_chamados_por_atendente \
        .withColumn("id_atendente", F.col("id_atendente").cast(IntegerType())) \
        .withColumn("nome_atendente", F.col("nome_atendente").cast(StringType())) \
        .withColumn("nivel_atendimento", F.col("nivel_atendimento").cast(IntegerType())) \
        .withColumn("total_chamados", F.col("total_chamados").cast(IntegerType())) \
        .withColumn("total_resolvidos", F.col("total_resolvidos").cast(IntegerType())) \
        .withColumn("nota_media_atendente", F.col("nota_media_atendente").cast(DecimalType(10,2))) \
        .withColumn("taxa_resolucao", F.col("taxa_resolucao").cast(DecimalType(10,2))) \
        .withColumn("processed_timestamp", F.col("processed_timestamp").cast(TimestampType()))

    df_chamados_por_atendente_typed = df_chamados_por_atendente_typed.select(
        "id_atendente", "nome_atendente", "nivel_atendimento",
        "total_chamados", "total_resolvidos", "taxa_resolucao",
        "nota_media_atendente", "processed_timestamp"
    )

    df_chamados_por_atendente_typed.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(tgt_table_chamados_por_atendente)

    gold_table_chamados_por_atendente = spark.table(tgt_table_chamados_por_atendente)
    final_count_chamados_por_atendente = gold_table_chamados_por_atendente.count()

    print(f"Salvo em: {tgt_table_chamados_por_atendente}")
    print(f"Contagem final: {final_count_chamados_por_atendente:,}")

    display(gold_table_chamados_por_atendente.limit(10))
else:
    print("Tabela df_atendentes_src não encontrada. Pulando criação de chamados_por_atendente.")

## Diagnóstico Operacional: Tempo e Eficiência por Nível de Atendimento

Agrega por `nivel_atendimento`:
- `tempo_medio_atendimento`, `tempo_medio_espera`: Medir gargalos operacionais  
- `taxa_resolucao_media`: Eficácia de cada nível  
- `total_chamados`: Volume absoluto por nível

Dados obtidos via join de `fact_chamados` com `ft_atendentes` (mapping chamado→nível), agregação, e cruzamento com métricas de resolução de `chamados_por_atendente`.

**Aplicação:** Identificar níveis gargalos (tempo > SLA ou taxa < meta).
Útil para dimensionamento de equipe, roteamento inteligente de chamadas, e definição de SLAs diferenciados por nível.
**Dependência:** Requer `ft_atendentes` e `chamados_por_atendente` processados.

In [0]:
tgt_table_tempo_por_nivel = f"{catalog_name}.{gold_db_name}.tempo_por_nivel"

if df_atendentes_src is not None and 'df_chamados_por_atendente_typed' in locals():
    taxa_por_nivel = df_chamados_por_atendente_typed.groupBy("nivel_atendimento").agg(
        F.round(F.avg("taxa_resolucao"), 2).alias("taxa_resolucao_media"),
        F.count("id_atendente").alias("total_atendentes")
    ).orderBy("nivel_atendimento")

    df_chamados_com_nivel = fact_chamados.join(
        df_atendentes_src, "id_atendente", "inner"
    ).select(
        "id_chamado", "id_atendente", "nivel_atendimento",
        "tempo_atendimento_minutos", "tempo_espera_minutos"
    )

    df_tempo_por_nivel = df_chamados_com_nivel.groupBy("nivel_atendimento").agg(
        F.round(F.avg("tempo_atendimento_minutos"), 2).alias("tempo_medio_atendimento"),
        F.round(F.avg("tempo_espera_minutos"), 2).alias("tempo_medio_espera"),
        F.count("id_chamado").alias("total_chamados")
    ).orderBy("nivel_atendimento")

    df_tempo_por_nivel_final = df_tempo_por_nivel.join(
        taxa_por_nivel.select("nivel_atendimento", "taxa_resolucao_media"),
        on="nivel_atendimento",
        how="left"
    ).orderBy("nivel_atendimento")

    df_tempo_por_nivel_final_typed = df_tempo_por_nivel_final \
        .withColumn("nivel_atendimento", F.col("nivel_atendimento").cast(IntegerType())) \
        .withColumn("tempo_medio_atendimento", F.col("tempo_medio_atendimento").cast(DecimalType(10,2))) \
        .withColumn("tempo_medio_espera", F.col("tempo_medio_espera").cast(DecimalType(10,2))) \
        .withColumn("taxa_resolucao_media", F.col("taxa_resolucao_media").cast(DecimalType(10,2))) \
        .withColumn("total_chamados", F.col("total_chamados").cast(IntegerType())) \
        .withColumn("processed_timestamp", F.current_timestamp().cast(TimestampType()))

    df_tempo_por_nivel_final_typed.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(tgt_table_tempo_por_nivel)

    gold_table_tempo_por_nivel = spark.table(tgt_table_tempo_por_nivel)
    final_count_tempo_por_nivel = gold_table_tempo_por_nivel.count()

    print(f"Salvo em: {tgt_table_tempo_por_nivel}")
    print(f"Contagem final: {final_count_tempo_por_nivel:,}")

    display(gold_table_tempo_por_nivel.limit(10))
else:
    print("Dependências não encontradas - pulando tempo_por_nivel")
    final_count_tempo_por_nivel = 0

## Análise: Satisfação por Perfil Demográfico de Cliente

Enriquece chamados com dados demográficos (`idade`, `regiao` de `dm_clientes`).
Calcula `faixa_etaria`: Até 24 | 25-44 | 45-64 | 65+ | Não informado.
Adiciona granularidade temporal (`ano`, `mes`, `dia`, `semana`).

**Correção crítica aplicada:** Join com `dm_clientes` APENAS para demográficos—satisfação (`nota_atendimento`) vem 100% de `fact_chamados`.
Elimina join redundante com Silver `ft_pesquisa_satisfacao`, garantindo 1:1 mapping.

**Aplicação:** Análises de NPS/CSAT por faixa etária e região—identifica segmentos com experiência degradada.

In [0]:
tgt_table_client_satisfaction = f"{catalog_name}.{gold_db_name}.ft_cliente_satisfacao"

if df_clientes_src is not None:
    df_client_satisfaction = fact_chamados.filter(
        col("nota_atendimento").isNotNull()
    ).alias("f").join(
        df_clientes_src.alias("c"),
        col("f.id_cliente") == col("c.id_cliente"),
        "inner"
    ).select(
        col("f.id_chamado"),
        col("f.id_cliente"),
        col("c.idade"),
        col("c.regiao"),
        col("f.nota_atendimento"),
        col("f.data")
    ).withColumn(
        "faixa_etaria",
        when(col("idade") < 25, "Até 24")
        .when((col("idade") >= 25) & (col("idade") <= 44), "25 a 44")
        .when((col("idade") >= 45) & (col("idade") <= 64), "45 a 64")
        .when(col("idade") >= 65, "65+")
        .otherwise("Não informado")
    ).withColumn("ano", year("data")) \
     .withColumn("mes", month("data")) \
     .withColumn("dia", dayofmonth("data")) \
     .withColumn("semana", weekofyear("data")) \
     .withColumn("processed_timestamp", current_timestamp())

    df_client_satisfaction_typed = df_client_satisfaction \
        .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
        .withColumn("id_cliente", col("id_cliente").cast(StringType())) \
        .withColumn("idade", col("idade").cast(IntegerType())) \
        .withColumn("regiao", col("regiao").cast(StringType())) \
        .withColumn("nota_atendimento", col("nota_atendimento").cast(IntegerType())) \
        .withColumn("faixa_etaria", col("faixa_etaria").cast(StringType())) \
        .withColumn("data", col("data").cast(DateType())) \
        .withColumn("ano", col("ano").cast(IntegerType())) \
        .withColumn("mes", col("mes").cast(IntegerType())) \
        .withColumn("dia", col("dia").cast(IntegerType())) \
        .withColumn("semana", col("semana").cast(IntegerType())) \
        .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

    df_client_satisfaction_typed.write.format("delta") \
        .mode("overwrite") \
        .option("overwriteSchema", "true") \
        .saveAsTable(tgt_table_client_satisfaction)

    gold_table_client_satisfaction = spark.table(tgt_table_client_satisfaction)
    final_count_client_satisfaction = gold_table_client_satisfaction.count()

    print(f"Salvo em: {tgt_table_client_satisfaction}")
    print(f"Contagem final: {final_count_client_satisfaction:,}")

    display(gold_table_client_satisfaction.limit(10))
else:
    print("Tabela dm_clientes não encontrada - pulando ft_cliente_satisfacao")
    final_count_client_satisfaction = 0

## Análise: Satisfação por Motivo ao Longo do Tempo

Agrega por `id_motivo`, `motivo` e granularidade temporal (`data`, `semana`, `ano`).
Calcula `nota_media` e métricas de satisfação/insatisfação por motivo-período.

**Correção aplicada:** Usa `nota_atendimento` de `fact_chamados` (não join com Silver)—garante 1:1 mapping.
**Aplicação:** Identifica motivos com tendência crescente de insatisfação (qualidade de resolução degradada, evolução de demanda complexa).
Útil para redesenho de fluxo de atendimento, treinamento de equipes, e priorização de melhorias operacionais.

In [0]:
tgt_table_reason_satisfaction = f"{catalog_name}.{gold_db_name}.ft_motivo_satisfacao"

df_reason_satisfaction = fact_chamados.filter(
    col("nota_atendimento").isNotNull()
).select(
    col("id_chamado"),
    col("id_motivo"),
    col("motivo"),
    col("nota_atendimento"),
    col("data")
).withColumn("ano", year("data")) \
 .withColumn("mes", month("data")) \
 .withColumn("dia", dayofmonth("data")) \
 .withColumn("semana", weekofyear("data")) \
 .withColumn("processed_timestamp", current_timestamp())

df_reason_satisfaction_typed = df_reason_satisfaction \
    .withColumn("id_chamado", col("id_chamado").cast(IntegerType())) \
    .withColumn("id_motivo", col("id_motivo").cast(IntegerType())) \
    .withColumn("motivo", col("motivo").cast(StringType())) \
    .withColumn("nota_atendimento", col("nota_atendimento").cast(IntegerType())) \
    .withColumn("data", col("data").cast(DateType())) \
    .withColumn("ano", col("ano").cast(IntegerType())) \
    .withColumn("mes", col("mes").cast(IntegerType())) \
    .withColumn("dia", col("dia").cast(IntegerType())) \
    .withColumn("semana", col("semana").cast(IntegerType())) \
    .withColumn("processed_timestamp", col("processed_timestamp").cast(TimestampType()))

df_reason_satisfaction_typed.write.format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable(tgt_table_reason_satisfaction)

gold_table_reason_satisfaction = spark.table(tgt_table_reason_satisfaction)
final_count_reason_satisfaction = gold_table_reason_satisfaction.count()

print(f"Salvo em: {tgt_table_reason_satisfaction}")
print(f"Contagem final: {final_count_reason_satisfaction:,}")

display(gold_table_reason_satisfaction.limit(10))

## Relatório Final: Sumário de Transformação

Exibe contagem de registros em todas as 14 tabelas Gold criadas:
- **8 Originais:** fact_chamados, funnel_metrics, anomalous_escalations, top_reasons_volume/cost/low_satisfaction, satisfaction_trends, client_profiles  
- **6 Analíticas:** satisfaction_correlations, channel_satisfaction, reason_satisfaction, client_satisfaction, chamados_por_atendente, tempo_por_nivel

**Validação:** Tabelas com contagem zero indicam problema em join/filtro ou ausência de dados—escale para troubleshooting.
Timestamp de processamento permite rastreamento de execução.

In [0]:
print("="*80)
print("RELATÓRIO FINAL DE TRANSFORMAÇÃO - CAMADA GOLD")
print("="*80)
print(f"Tabelas Gold criadas: 14")
print(f"\nTabelas Originais:")
print(f"  - funnel_metrics: {final_count_funnel:,} registros")
print(f"  - anomalous_escalations: {final_count_anomalous:,} registros")
print(f"  - top_reasons_volume: {final_count_top_volume:,} registros")
print(f"  - top_reasons_cost: {final_count_top_cost:,} registros")
print(f"  - top_reasons_low_satisfaction: {final_count_top_satisfaction:,} registros")
print(f"  - satisfaction_trends: {final_count_satisfaction_trends:,} registros")
print(f"  - client_profiles: {final_count_client_profiles:,} registros")
print(f"\nTabelas Adicionais de Análise:")
print(f"  - satisfaction_correlations: {final_count_satisfaction_correlations:,} registros")
print(f"  - channel_satisfaction: {final_count_channel_satisfaction:,} registros")
print(f"  - reason_satisfaction: {final_count_reason_satisfaction:,} registros")
print(f"  - client_satisfaction: {final_count_client_satisfaction:,} registros")
print(f"  - chamados_por_atendente: {final_count_chamados_por_atendente:,} registros")
print(f"  - tempo_por_nivel: {final_count_tempo_por_nivel:,} registros")
print("="*80)
print(f"Timestamp de processamento: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

In [0]:
spark.sql("""
CREATE VOLUME IF NOT EXISTS atendimento_catalog.default.exports
""")

spark.sql("""
CREATE VOLUME IF NOT EXISTS atendimento_catalog.default.exports_tmp
""")

export_base = "/Volumes/atendimento_catalog/default/exports"
tmp_base = "/Volumes/atendimento_catalog/default/exports_tmp"

dbutils.fs.mkdirs(tmp_base)

tabelas = [
    "anomalous_escalations",
    "chamados_por_atendente",
    "client_profiles",
    "fact_chamados",
    "ft_canal_satisfacao",
    "ft_cliente_satisfacao",
    "ft_correlacao_satisfacao",
    "ft_motivo_satisfacao",
    "funnel_metrics",
    "satisfaction_trends",
    "tempo_por_nivel",
    "top_reasons_cost",
    "top_reasons_low_satisfaction",
    "top_reasons_volume"
]

for tabela in tabelas:

    tabela_full = f"{catalog_name}.{gold_db_name}.{tabela}"
    tmp_path = f"{tmp_base}/{tabela}"
    final_file = f"{export_base}/{tabela}.csv"
    
    dbutils.fs.rm(tmp_path, recurse=True)

    spark.table(tabela_full) \
         .coalesce(1) \
         .write \
         .option("header", "true") \
         .mode("overwrite") \
         .csv(tmp_path)

    files = dbutils.fs.ls(tmp_path)
    part_file = [f.path for f in files if f.path.endswith(".csv")][0]
    dbutils.fs.cp(part_file, final_file)
    dbutils.fs.rm(tmp_path, recurse=True)

spark.sql("""
DROP VOLUME atendimento_catalog.default.exports_tmp
""")


In [ ]:
# 1. Carregar tabelas de origem

df_fact = spark.table("gold.fact_chamados")

try:
    df_motivos = spark.table("silver.dm_motivos")
except:
    df_motivos = None

try:
    df_atendentes = spark.table("silver.ft_atendentes")
except:
    df_atendentes = None


# 2. Join com dimensões

df_gold = df_fact

# 2.1 Join com motivos
if df_motivos is not None:
    df_gold = (
        df_gold.alias("f")
        .join(
            df_motivos.alias("m"),
            F.col("f.id_motivo") == F.col("m.id_motivo"),
            "left"
        )
        .select(
            "f.*",
            F.col("m.nome_motivo"),
            F.col("m.categoria")
        )
    )
else:
    df_gold = (
        df_gold
        .withColumn("nome_motivo", F.lit(None).cast(StringType()))
        .withColumn("categoria", F.lit(None).cast(StringType()))
    )

# 2.2 Join com atendentes (nível)
if df_atendentes is not None:
    df_gold = (
        df_gold.alias("f")
        .join(
            df_atendentes.alias("a"),
            F.col("f.id_atendente") == F.col("a.id_atendente"),
            "left"
        )
        .select(
            "f.*",
            F.col("a.nivel_atendimento")
        )
    )
else:
    df_gold = df_gold.withColumn("nivel_atendimento", F.lit(None).cast("int"))


# 3. Preparação dos Dados

df_gold = (
    df_gold
    .withColumnRenamed("data", "data_chamado")
    .withColumn("avg_tempo_espera_seconds", F.col("tempo_espera_minutos") * 60)
    .withColumn("avg_tempo_atendimento_seconds", F.col("tempo_atendimento_minutos") * 60)
    .withColumn("resolvido_int", F.when(F.col("resolvido") == "Sim", 1).otherwise(0))
    .withColumn("escalado_nivel2_int", F.when(F.col("nivel_atendimento") == 2, 1).otherwise(0))
)


# 4. Agregações por Data e Canal

df_metrics = (
    df_gold.groupBy("data_chamado", "canal")
    .agg(
        F.count("*").alias("total_chamados"),
        F.sum("resolvido_int").alias("resolvidos"),
        F.avg("resolvido_int").alias("taxa_resolucao"),
        F.avg("avg_tempo_espera_seconds").alias("avg_tempo_espera_seconds"),
        F.avg("avg_tempo_atendimento_seconds").alias("avg_tempo_atendimento_seconds"),
        F.avg("custo").alias("avg_custo"),
        F.avg("nota_atendimento").alias("avg_nota"),
        F.avg("escalado_nivel2_int").alias("pct_escalado_para_nivel2")
    )
)


# 5. Impacto de custo

df_metrics = df_metrics.withColumn(
    "impacto_custo",
    F.col("total_chamados") * F.col("avg_custo")
)


# 6A. Cálculo REAL do churn_risk_score_est

df_metrics = df_metrics.withColumn(
    "churn_risk_score_est",
    (
        0.3 * (F.col("avg_tempo_espera_seconds") / 1800) +
        0.4 * (1 - F.col("taxa_resolucao")) +
        0.3 * (1 - (F.col("avg_nota") / 5))
    ).cast(DecimalType(18,6))
)


# 6B. Cálculo dos Top 3 Motivos

df_motivos_agg = (
    df_gold.groupBy("data_chamado", "canal", "nome_motivo")
           .agg(F.count("*").alias("qtd"))
)

w = Window.partitionBy("data_chamado", "canal").orderBy(F.desc("qtd"))

df_top3 = (
    df_motivos_agg
    .withColumn("rn", F.row_number().over(w))
    .filter("rn <= 3")
    .groupBy("data_chamado", "canal")
    .agg(F.collect_list("nome_motivo").alias("top3_motivos"))
)

df_metrics = (
    df_metrics.alias("m")
    .join(df_top3.alias("t"), ["data_chamado", "canal"], "left")
    .select("m.*", "t.top3_motivos")
)


# 7. Tipagem Final

df_metrics = (
    df_metrics
    .withColumn("data_chamado", F.col("data_chamado").cast(DateType()))
    .withColumn("avg_tempo_espera_seconds", F.col("avg_tempo_espera_seconds").cast(DecimalType(18, 6)))
    .withColumn("avg_tempo_atendimento_seconds", F.col("avg_tempo_atendimento_seconds").cast(DecimalType(18, 6)))
    .withColumn("avg_custo", F.col("avg_custo").cast(DecimalType(18, 6)))
    .withColumn("avg_nota", F.col("avg_nota").cast(DecimalType(18, 6)))
    .withColumn("impacto_custo", F.col("impacto_custo").cast(DecimalType(18, 6)))
    .withColumn("pct_escalado_para_nivel2", F.col("pct_escalado_para_nivel2").cast(DecimalType(18, 6)))
    .withColumn("churn_risk_score_est", F.col("churn_risk_score_est").cast(DecimalType(18, 6)))
    .withColumn("top3_motivos", F.col("top3_motivos").cast("array<string>"))
)


# 8. Escrever tabela GOLD

df_metrics.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.channel_performance")

print("Criado: gold.channel_performance")
print(f"Registros: {df_metrics.count():,}")

## Análise: Métricas de Chamados por Data e Canal

Agrega por `data_chamado` e `canal`, calculando métricas de volume, tempo, custo, satisfação e escalonamento.

**Métricas principais:**
- `total_chamados` / `resolvidos` / `taxa_resolucao` → volume e eficiência do atendimento
- `avg_tempo_espera_seconds` / `avg_tempo_atendimento_seconds` → tempo médio de resposta e atendimento
- `avg_custo` / `impacto_custo` → custo médio e impacto financeiro
- `avg_nota` → satisfação média dos clientes
- `pct_escalado_para_nivel2` → percentual de chamados que precisaram escalar para nível 2
- `churn_risk_score_est` → estimativa de risco de churn baseada em tempo de espera, taxa de resolução e nota de atendimento
- `top3_motivos` → motivos mais frequentes/impactantes por canal e data

**Correção aplicada:** Joins com dimensões `dm_motivos` e `ft_atendentes`, conversão de tempo para segundos e cálculo real de churn/ranking dos top motivos.  

**Aplicação:** Identifica gargalos operacionais, motivos de maior impacto em custo e satisfação, permitindo ações de melhoria no atendimento, treinamento de equipes e monitoramento financeiro.

In [ ]:
# 1. Carregar tabelas de origem

df_fact = spark.table("gold.fact_chamados")

try:
    df_motivos = spark.table("silver.dm_motivos")
except Exception as e:
    df_motivos = None


# 2. Join com dimensão motivo

df_gold = df_fact

if df_motivos is not None:
    df_gold = (
        df_gold.alias("f")
        .join(
            df_motivos.alias("m"),
            F.col("f.id_motivo") == F.col("m.id_motivo"),
            "inner"
        )
        .select(
            "f.*",
            F.col("m.nome_motivo"),
            F.col("m.categoria")
        )
    )
else:
    df_gold = (
        df_gold
        .withColumn("nome_motivo", F.lit(None).cast(StringType()))
        .withColumn("categoria", F.lit(None).cast(StringType()))
    )


# 3. Conversão de data

df_gold = df_gold.withColumnRenamed("data", "data_chamado")


# 4. Agregações por motivo/categoria

df_metrics = (
    df_gold.groupBy(
        "data_chamado",
        "id_motivo",
        "nome_motivo",
        "categoria"
    )
    .agg(
        F.count("*").alias("total_chamados"),
        F.avg("tempo_atendimento_minutos").alias("avg_tempo_atendimento"), 
        F.avg("custo").alias("avg_custo"),
        F.avg("nota_atendimento").alias("avg_nota"),
        F.avg(F.when(F.col("resolvido") == "Sim", 1).otherwise(0)).alias("taxa_resolucao")
    )
)


# 5. Impacto de custo

df_metrics = df_metrics.withColumn(
    "impacto_custo",
    F.col("total_chamados") * F.col("avg_custo")
)


# 6. Tipagem final

df_metrics = (
    df_metrics
    .withColumn("data_chamado", F.col("data_chamado").cast(DateType()))
    .withColumn("id_motivo", F.col("id_motivo").cast(StringType()))
    .withColumn("nome_motivo", F.col("nome_motivo").cast(StringType()))
    .withColumn("categoria", F.col("categoria").cast(StringType()))
    .withColumn("total_chamados", F.col("total_chamados").cast("int"))
    .withColumn("taxa_resolucao", F.col("taxa_resolucao").cast(DecimalType(18, 6)))
    .withColumn("avg_tempo_atendimento", F.col("avg_tempo_atendimento").cast(DecimalType(18, 6)))
    .withColumn("avg_custo", F.col("avg_custo").cast(DecimalType(18, 6)))
    .withColumn("avg_nota", F.col("avg_nota").cast(DecimalType(18, 6)))
    .withColumn("impacto_custo", F.col("impacto_custo").cast(DecimalType(18, 6)))
)


# 7. Escrever tabela GOLD

df_metrics.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.reason_metrics")

## Análise: Métricas de Chamados por Motivo e Categoria

Agrega por `data_chamado`, `id_motivo`, `nome_motivo` e `categoria`.
Calcula `total_chamados`, `avg_tempo_atendimento`, `avg_custo`, `avg_nota` e `taxa_resolucao`.

**Correção aplicada:** Usa join com `dm_motivos` e mapeamento direto de `nota_atendimento` da tabela fato—garante 1:1 mapping de chamados e notas.
**Aplicação:** Identifica motivos/categorias com maior volume, custo ou impacto na satisfação.
Útil para monitoramento de desempenho de atendimento, análise de eficiência operacional e priorização de melhorias por tipo de chamado.

In [ ]:
# 1. Ler tabelas

df_fact = spark.table("gold.fact_chamados")

print("Linhas com data nula na fato:", df_fact.filter(F.col("data").isNull()).count())

try:
    df_motivos = spark.table("silver.dm_motivos")
except:
    df_motivos = None


# 2. Join com dimensão motivo

df_gold = df_fact

if df_motivos is not None:
    df_gold = (
        df_gold.alias("f")
        .join(
            df_motivos.alias("m"),
            F.col("f.id_motivo") == F.col("m.id_motivo"),
            "left"
        )
        .select("f.*", F.col("m.nome_motivo"))
    )
else:
    df_gold = df_gold.withColumn(
        "nome_motivo", F.lit(None).cast(StringType())
    )


# 3. Filtrar custos válidos

df_gold = (
    df_gold
    .filter(F.col("flag_custo_ausente") == 0)
    .filter(F.col("nome_motivo").isNotNull())
)


# 4. Criar data como date

df_gold = df_gold.withColumn("data", F.col("data").cast("date"))


# 5. Agregações principais

df_agg = (
    df_gold.groupBy("data", "canal")
    .agg(
        F.sum("custo").alias("custo_total"),
        F.avg("custo").alias("custo_medio_por_chamado")
    )
)


# 6. Ranking dos motivos por custo

window_motivo = Window.partitionBy("data", "canal").orderBy(F.desc("custo_motivo"))

df_motivos_rank = (
    df_gold.groupBy("data", "canal", "nome_motivo")
    .agg(F.sum("custo").alias("custo_motivo"))
    .withColumn("rn", F.row_number().over(window_motivo))
)

df_motivos_top = (
    df_motivos_rank.filter(F.col("rn") <= 3)
    .groupBy("data", "canal")
    .agg(
        F.collect_list("nome_motivo").alias("top_motivos_por_custo")
    )
)


# 7. Join final

df_final = df_agg.join(df_motivos_top, on=["data", "canal"], how="left")

df_final = df_final.withColumn(
    "top_motivos_por_custo",
    F.when(F.col("top_motivos_por_custo").isNull(), F.array().cast("array<string>"))
     .otherwise(F.col("top_motivos_por_custo"))
)


# 8. Remover duplicações

df_final = df_final.dropDuplicates(["data", "canal"])


# 9. Tipagem final

df_final = (
    df_final
    .withColumn("data", F.col("data").cast("date"))
    .withColumn("canal", F.col("canal").cast(StringType()))
    .withColumn("custo_total", F.col("custo_total").cast(DecimalType(18, 8)))
    .withColumn("custo_medio_por_chamado", F.col("custo_medio_por_chamado").cast(DecimalType(18, 8)))
    .withColumn("top_motivos_por_custo", F.col("top_motivos_por_custo").cast("array<string>"))
)


# 10. Ordenar ANTES de salvar

df_final_sorted = df_final.orderBy(F.col("custo_total").desc())


# 11. Gravar tabela GOLD

df_final_sorted.write \
    .format("delta") \
    .mode("overwrite") \
    .option("overwriteSchema", "true") \
    .saveAsTable("gold.cost_summary")

print("Criado: gold.cost_summary")
print(f"Registros: {df_final_sorted.count():,}")